In [2]:
import json, glob
from dotenv import load_dotenv, find_dotenv; _ = load_dotenv(find_dotenv())

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [4]:
topic_type = 'foods'
topic_file = f'{topic_type}.txt'

In [5]:
with open(f'config/{topic_file}', 'r') as f:
    topic_list = f.read().splitlines()
topic_list

['じゃがいも', '玉ねぎ', 'にんじん', 'なす', 'トマト', 'ピーマン']

In [46]:
with open(f'config/config.json', 'r') as f:
    config = json.load(f)
config

{'language': {'Japanese': 'ja',
  'English': 'en',
  'Chinese': 'zh',
  'German': 'de',
  'Arabic': 'ar',
  'Russian': 'ru'},
 'format': {'prose': '',
  'bullet_list': '情報を構造化して箇条書きに変換してください。',
  'markdown': '階層構造を持つMarkdownに変換してください。各セクション内の文章は散文にしてください。',
  'json': '情報を構造化してJson形式に変換してください。Key-Valueは入力と同じ言語にしてください。',
  'convesation': 'AssistantからUserへ情報を伝える会話形式にしてください。'},
 'length': ['full', 'short_sum', 'long_sum']}

## Translate chain

In [7]:
template_str = """\
ユーザーが入力したテキストを指示された言語に翻訳してください。
翻訳テキストは元のテキストの情報を全て保持するようにしてください。
"""

In [8]:
prompt_template = ChatPromptTemplate.from_messages([
    ('system', template_str),
    ('user', '{text}'),
    ('user', '言語: {language}')
])

chat = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0., max_tokens=4096)

translate_chain = prompt_template | chat

### Test

In [9]:
# topic = topic_list[0]
# length_type = 'full'
# form = 'prose'

# res = translate_chain.batch([
#     {
#         'text': open(f'./data/{topic_type}/{topic}/ja-{form}-{length_type}.txt', 'r').read(),
#         'language': lang
#     }
#     for lang, lang2 in config['language'].items() if lang2 != 'ja'
# ])

In [10]:
# for r in res:
#     print(r.content)

### Batch

In [11]:
files = [
    file
    for topic in topic_list
    for file in glob.glob(f'./data/{topic_type}/{topic}/*.txt')
]
print(len(files))

90


In [47]:
variables = [
    {
        'file': file,
        'language': lang,
        'language2': lang2,
        'text': open(file,'r').read()
    }
    for lang, lang2 in config['language'].items() if lang2 not in ['ja','zh']
    for file in files
]

In [48]:
# res = translate_chain.batch([
#     {
#         'text': v['text'],
#         'language': v['language']
#     }
#     for v in variables
# ])

In [50]:
len(variables)

360

In [51]:
res = []

In [52]:
from tqdm import tqdm
for i,v in enumerate(tqdm(variables)):
    if len(res) > i:
        continue
    r = translate_chain.invoke({
        'text': v['text'],
        'language': v['language']
    }, config=config)
    res.append(r)

100%|██████████| 360/360 [47:18<00:00,  7.88s/it]


In [55]:
for r, v in zip(res, variables):
    # print(v['file'].replace('ja',v['language2']))
    with open(v['file'].replace('ja',v['language2']), 'w') as f:
        f.write(r.content)